In [9]:
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from skimage.measure import moments
from scipy.stats import describe

# Preprocessing
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (128, 128))
    _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return binary_image

# Feature Extraction
def extract_features(image):
    # HOG features
    hog_features = hog(image, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=False)

    # Local Binary Pattern (LBP) features
    lbp = local_binary_pattern(image, 8, 1, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), density=True)

    # Image moments
    m = moments(image)
    hu_moments = cv2.HuMoments(m).flatten()

    # Additional features
    aspect_ratio = image.shape[0] / image.shape[1]
    white_pixel_ratio = np.sum(image == 255) / image.size

    features = np.concatenate([hog_features, lbp_hist, hu_moments, [aspect_ratio, white_pixel_ratio]])
    return features

# Data Augmentation
def augment_image(image):
    augmented_images = []
    rows, cols = image.shape

    for angle in range(-10, 11, 5):  # Rotate the image slightly
        M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
        rotated = cv2.warpAffine(image, M, (cols, rows))
        augmented_images.append(rotated)

    return augmented_images

# Model Training
from sklearn.svm import OneClassSVM

original_image_path ='path/to/original_signature.jpg'
binary_image = preprocess_image(original_image_path)
augmented_images = augment_image(binary_image)
X = [extract_features(img) for img in augmented_images]

clf = OneClassSVM(gamma='auto').fit(X)
print("Model trained on augmented data")

# Signature Verification
def verify_signature(test_image_path, model):
    test_image = preprocess_image(test_image_path)
    test_features = extract_features(test_image)
    prediction = model.predict([test_features])
    
    # Print descriptive statistical features of the test image
    test_stats = describe(test_image.ravel())
    print("\nDescriptive Statistical Features of Test Image:")
    print("Mean:", test_stats.mean)
    print("Variance:", test_stats.variance)
    print("Skewness:", test_stats.skewness)
    print("Kurtosis:", test_stats.kurtosis)
    
    # Print descriptive statistical features of the original image
    original_image = preprocess_image(original_image_path)
    original_stats = describe(original_image.ravel())
    print("\nDescriptive Statistical Features of Original Image:")
    print("Mean:", original_stats.mean)
    print("Variance:", original_stats.variance)
    print("Skewness:", original_stats.skewness)
    print("Kurtosis:", original_stats.kurtosis)
    
    return "Genuine" if prediction == 1 else "Forged"

Model trained on augmented data


In [11]:
test_image_path ='path/to/test_signature.jpg'
result = verify_signature(test_image_path, clf)
print("The signature is:", result)


Descriptive Statistical Features of Test Image:
Mean: 248.22967529296875
Variance: 1680.6980853245777
Skewness: -5.889961141922817
Kurtosis: 32.69164225336075

Descriptive Statistical Features of Original Image:
Mean: 246.57989501953125
Variance: 2076.355332809585
Skewness: -5.226739482098385
Kurtosis: 25.318805613726088
The signature is: Forged
